In [11]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from assnat.clean import complete_preproc
from assnat.params import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle as pk

def simple_logistic_regression(legislation_number, merge_data_sets=True, drop_col = DROP_COLS, drop_fam = DROP_FAM, na_col = NA_COLS , drop_names = DROP_NAMES , min_words= MIN_WORDS, punct_opt=PUNCT_OPT):
    if merge_data_sets:
        data1 = pd.read_csv('data/leg15.csv')
        data2 = pd.read_csv('data/leg16.csv')
        df = pd.concat([data1, data2], ignore_index=True, axis=0)
    else:
        df = pd.read_csv(f'data/leg{legislation_number}.csv')

    df = complete_preproc(df, drop_col= drop_col, drop_fam= drop_fam, na_col= na_col, drop_names= drop_names, min_words = min_words, punct_opt= punct_opt)

    X_train, X_test, y_train, y_test = train_test_split(
        df[['Texte', 'Thème Séance']],
        df['famille'],
        test_size=0.2,
        random_state=42
    )

    tfidf_vectorizer = TfidfVectorizer(max_features=100000)

    preprocessor = ColumnTransformer(
        transformers=[
            ('tfidf_texte', tfidf_vectorizer, 'Texte'),
            ('tfidf_theme', tfidf_vectorizer, 'Thème Séance')
        ],
        remainder='passthrough'
    )

    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(multi_class='multinomial', max_iter=100000))
    ])

    model_pipeline.fit(X_train, y_train)

    y_pred = model_pipeline.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(y_test, y_pred))
    pk.dump(model_pipeline, open('/Users/aaronviviani/code/aaronviviani/08\ -\ Project ', 'wb'))
    return accuracy

simple_logistic_regression(15, merge_data_sets= True, drop_fam= ['Variable'], na_col = ['Texte','famille','Nom Orateur'], drop_names = ['Mme la présidente', 'M. le président'], min_words = 75, punct_opt= True)


Columns dropped
family dropped
NaN dropped
Names dropped
Short sentences removed
Preprocessing done!


/Users/aaronviviani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.4939667217071002
                precision    recall  f1-score   support

        Centre       0.52      0.81      0.63      6056
  Centre-droit       0.37      0.11      0.17      1452
 Centre-gauche       0.55      0.05      0.10       433
        Droite       0.47      0.37      0.41      2700
Extrême droite       0.52      0.17      0.25       486
Extrême gauche       0.50      0.39      0.44      1870
        Gauche       0.43      0.33      0.37      2749

      accuracy                           0.49     15746
     macro avg       0.48      0.32      0.34     15746
  weighted avg       0.48      0.49      0.46     15746



0.4939667217071002

In [53]:
simple_logistic_regression(15, merge_data_sets= True, drop_fam= ['Variable'], na_col = ['Texte','famille','Nom Orateur'], drop_names = ['Mme la présidente', 'M. le président'], min_words = 75, punct_opt= True)


Columns dropped
family dropped
NaN dropped
Names dropped
Short sentences removed
Preprocessing done!


/Users/aaronviviani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.4939667217071002
                precision    recall  f1-score   support

        Centre       0.52      0.81      0.63      6056
  Centre-droit       0.37      0.11      0.17      1452
 Centre-gauche       0.55      0.05      0.10       433
        Droite       0.47      0.37      0.41      2700
Extrême droite       0.52      0.17      0.25       486
Extrême gauche       0.50      0.39      0.44      1870
        Gauche       0.43      0.33      0.37      2749

      accuracy                           0.49     15746
     macro avg       0.48      0.32      0.34     15746
  weighted avg       0.48      0.49      0.46     15746



0.4939667217071002

In [29]:
from transformers import AutoTokenizer, FlaubertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_base_cased")
model = FlaubertModel.from_pretrained("flaubert/flaubert_base_cased")

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).